In [54]:
import pickle
from datetime import datetime
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.engine import url

In [55]:
# pyodbc stuff for MS SQL Server Express
driver='{SQL Server}'
server='XX.XX.XX.XX'
database='CDR_DM'
username = 'DADevUser'
password = 'XXXXX'
trusted_connection='yes'
# pyodbc connection string
connection_string = f'DRIVER={driver};SERVER={server};'
connection_string += f'DATABASE={database};'
connection_string += f'UID={username};'
connection_string += f'PWD={password};'
#connection_string += f'TRUSTED_CONNECTION={trusted_connection}'
# create sqlalchemy engine connection URL
connection_url = url.URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = sqlalchemy.create_engine(connection_url)

In [56]:
connection = engine.connect()
metadata = sqlalchemy.MetaData()
census = sqlalchemy.Table('ML_Centene_IO_Data_Test', metadata, autoload=True, autoload_with=engine)

In [57]:
query= sqlalchemy.select([census.columns.UCRN,
census.columns.BillType,
census.columns.Header_PatientAge,
census.columns.Header_LOS,
census.columns.Header_PatientGender,
census.columns.Header_PrincipalDiagnosisCode,
census.columns.Header_SD1,
census.columns.Header_SD2,
census.columns.Header_SD3,
census.columns.Header_SD4,
census.columns.Header_SD5,
census.columns.Header_PriorAuthorizationPresent,
census.columns.Header_IsCharged,
census.columns.Header_IsAllowed,
census.columns.Header_IsPaid,
census.columns.Header_ConditionCodes1,
census.columns.Header_OccurrenceCodes1,
census.columns.Line_HCPCS_CPTCode,
census.columns.Line_IsModifierAvailable,
census.columns.Header_Quantity,
census.columns.Line_LOS,
census.columns.Line_IsCharged,
census.columns.Line_IsAllowed,
census.columns.Line_IsPaid,
census.columns.IsHcpcsAvlOnOthLineWtSameDtWithMod,
census.columns.IsHcpcsAvlOnOthLineWtSameDtWoMod,
census.columns.IsHcpcsAvlOnOthLineDiffDtWithMod,
census.columns.IsHcpcsAvlOnOthLineDiffDtWoMod,
census.columns.OutCome])

In [58]:
#ResultProxy = connection.execute(query.limit(100))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [59]:
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df_Org = df[['UCRN','OutCome']]


In [60]:
df.drop(columns=['UCRN'],inplace=True)
df.drop(columns=['OutCome'],inplace=True)
df = df.replace({False: 0, True:1})

In [61]:
df.head()

,BillType,Header_PatientAge,Header_LOS,Header_PatientGender,Header_PrincipalDiagnosisCode,Header_SD1,Header_SD2,Header_SD3,Header_SD4,Header_SD5,...,Line_IsModifierAvailable,Header_Quantity,Line_LOS,Line_IsCharged,Line_IsAllowed,Line_IsPaid,IsHcpcsAvlOnOthLineWtSameDtWithMod,IsHcpcsAvlOnOthLineWtSameDtWoMod,IsHcpcsAvlOnOthLineDiffDtWithMod,IsHcpcsAvlOnOthLineDiffDtWoMod
0,131,61,1,F,Z1231,Z1231,Z1231,Z1231,Z1231,Z1231,...,0,1,1,1,1,1,0,1,0,0
1,131,61,1,F,Z1231,Z1231,Z1231,Z1231,Z1231,Z1231,...,0,1,1,1,1,1,0,1,0,0
2,131,53,1,F,G8929,G8929,G8929,G8929,G8929,G8929,...,0,1,1,1,1,1,0,0,0,0
3,131,26,1,F,J029,K120,J029,J029,J029,J029,...,0,12,1,1,1,1,0,1,0,0
4,131,5,1,F,A389,A389,A389,A389,A389,A389,...,0,1,1,1,1,1,1,1,0,0


In [62]:
test_input = df.to_numpy()

In [63]:
#test_input.shape

In [64]:
pipe = pickle.load(open('Ovrpay_IO_Model_rf.pkl','rb'))
test_pred = pipe.predict(test_input)

In [65]:
rf_out = pd.concat([df_Org, pd.DataFrame(test_pred)], axis = 1)
date = datetime.now().strftime("%Y%m%d%I%M%S")
prediction = pd.DataFrame(rf_out, columns=['UCRN',0]).to_csv('Ovrpay_IO_Model_rf_pred'+date+'.csv')

In [66]:
#rf_out.head()